In [3]:
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [4]:
point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

True


In [5]:
type(polygon)

shapely.geometry.polygon.Polygon

In [6]:
base = "/Users/macbook/Documents/BTP/Satellite/Data/Maps/Districts/Census_2011"

In [7]:
sf = shapefile.Reader(base+"/2011_Dist.shp")

In [28]:
type(sf)
print sf.fields

[('DeletionFlag', 'C', 1, 0), ['DISTRICT', 'C', 28, 0], ['ST_NM', 'C', 24, 0], ['ST_CEN_CD', 'N', 9, 0], ['DT_CEN_CD', 'N', 9, 0], ['censuscode', 'N', 14, 0]]


In [37]:
rec = sf.records()
rec

[['Adilabad', 'Andhra Pradesh', 28, 1, 532],
 ['Agra', 'Uttar Pradesh', 9, 15, 146],
 ['Ahmadabad', 'Gujarat', 24, 7, 474],
 ['Ahmadnagar', 'Maharashtra', 27, 26, 522],
 ['Aizawl', 'Mizoram', 15, 3, 283],
 ['Ajmer', 'Rajasthan', 8, 21, 119],
 ['Akola', 'Maharashtra', 27, 5, 501],
 ['Alappuzha', 'Kerala', 32, 11, 598],
 ['Aligarh', 'Uttar Pradesh', 9, 12, 143],
 ['Alirajpur', 'Madhya Pradesh', 23, 48, 465],
 ['Allahabad', 'Uttar Pradesh', 9, 44, 175],
 ['Almora', 'Uttarakhand', 5, 9, 64],
 ['Alwar', 'Rajasthan', 8, 6, 104],
 ['Ambala', 'Haryana', 6, 2, 70],
 ['Ambedkar Nagar', 'Uttar Pradesh', 9, 47, 178],
 ['Amravati', 'Maharashtra', 27, 7, 503],
 ['Amreli', 'Gujarat', 24, 13, 480],
 ['Amritsar', 'Punjab', 3, 15, 49],
 ['Anand', 'Gujarat', 24, 15, 482],
 ['Anantapur', 'Andhra Pradesh', 28, 22, 553],
 ['Anantnag', 'Jammu & Kashmir', 1, 14, 14],
 ['Anjaw', 'Arunanchal Pradesh', 12, 16, 260],
 ['Anugul', 'Odisha', 21, 15, 384],
 ['Anuppur', 'Madhya Pradesh', 23, 44, 461],
 ['Araria', 'Bih

In [9]:
shapes = sf.shapes()

In [10]:
type(shapes)

list

In [11]:
len(shapes)

641

In [12]:
shapes[0]

<shapefile._Shape instance at 0x10ed25248>

In [18]:
for name in dir(shapes[3]):
    if not name.startswith('__'):
        print(name)

bbox
parts
points
shapeType


In [19]:
print(type(shapes[3].bbox))
print(type(shapes[3].parts))
print(type(shapes[3].points))
print(type(shapes[3].shapeType))

<class 'shapefile._Array'>
<class 'shapefile._Array'>
<type 'list'>
<type 'int'>


In [25]:
print(shapes[3].bbox)
print(shapes[3].parts)
print(shapes[3].points)
print(shapes[3].shapeType)

[73.62021077854777, 18.330751971624714, 75.58884115831344, 19.987712875559378]
[0]
[(74.67332670904928, 19.946696343556695), (74.67392695152176, 19.93509492197915), (74.68201678696512, 19.932224850721763), (74.68195053074005, 19.924933850115533), (74.65599350186761, 19.922162914855576), (74.65309980701223, 19.919389609985174), (74.64916140192294, 19.909825891926886), (74.64999277204926, 19.88634427465867), (74.64710694859035, 19.87323657862446), (74.64809925517882, 19.861238708397742), (74.6633289457034, 19.861116434008252), (74.66086700405754, 19.839049149236022), (74.64472026986222, 19.840892223128517), (74.63512165837304, 19.843693978680033), (74.63279965439683, 19.845058666045166), (74.60668541954195, 19.84963475868926), (74.60427319260945, 19.821595418496592), (74.6014517979137, 19.801643547234157), (74.59197046327047, 19.786992836196717), (74.59165682805882, 19.77969844654881), (74.59338261029224, 19.77648982074721), (74.59783153387845, 19.773021451890557), (74.60197320421331, 19

In [30]:
point = Point(74.62399910168892, 19.785850073716652)
polygon = Polygon(shapes[0].points)
print(polygon.contains(point))

False
